In [1]:
from xml.etree import ElementTree
from xml.dom import minidom
import glob, os, sys, time
from shutil import copy
from xml.etree.ElementTree import Element, SubElement, Comment
import subprocess as sp
import pandas as pd
from datetime import datetime
from osgeo import ogr
import logging
import re
non_decimal = re.compile(r'[^\d.,-]+')

In [2]:
def get_features(shape):
    driver = ogr.GetDriverByName('ESRI Shapefile')

    dataSource = driver.Open(shape, 0) # 0 means read-only. 1 means writeable.

    # Check to see if shapefile is found.
    if dataSource is None:
        logger.error ('Could not open %s' % (shape))
        return
    else:
        logger.info ('Opened %s' % (shape))
        layer = dataSource.GetLayer()
        shape_features = layer.GetFeatureCount()
        logger.info ('Name of layer: %s' % layer.GetDescription())
        logger.info ("Number of features in %s: %d" % (os.path.basename(shape),shape_features))

        features_shape = []
        for i in range(shape_features):
            feat = layer.GetFeature(i)
            obID = feat.GetField('OBJECTID')
            features_shape.append(obID)
#         logger.info (obID)
        return dataSource, layer, features_shape

In [3]:
def prettify(elem):
    """Return a pretty-printed XML string for the Element.
    """
    rough_string = ElementTree.tostring(elem, 'utf-8')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="  ")

def writeOGRVRT(basename, fullpath):
    top = Element('OGRVRTDataSource')
    child = SubElement(top, 'OGRVRTLayer')
    child.set('name', basename)
    sub_child_1 = SubElement(child, 'SrcDataSource')
    sub_child_1.text = fullpath
    sub_child_2 = SubElement(child, 'GeometryType')
    sub_child_2.text = 'wkbPoint'
    sub_child_3 = SubElement(child, 'LayerSRS')
    sub_child_3.text = 'EPSG:28992'
    sub_child_4 = SubElement(child, 'GeometryField')
    sub_child_4.set('encoding','PointFromColumns')
    sub_child_4.set('x','field_1')
    sub_child_4.set('y','field_2')
    sub_child_4.set('z','field_3')

    return prettify(top)

In [6]:
rootDir = r'D:\jupyter\BodempeilingScript_v4\BodempeilingScript_v4'
ruweDataDir = os.path.join(rootDir, 'survey_ruweData')
asciiDataDir = os.path.join(rootDir, 'survey_asciiData')
workdir = os.path.join(rootDir, 'tmp')
y = os.path.join(rootDir, 'achtergrondShp\Achtergrond_polygonen.shp')
logFile = os.path.join(rootDir, 'log\log_file.out')
xmldir = os.path.join(rootDir, 'xml')

In [21]:
rootDir = r'D:\OMS_Waddenzee\trunk\fews'
ruweDataDir = os.path.join(rootDir, r'Import\rws\bodempeilingen_historisch\ruweData')
ruweDataDirBackup = os.path.join(rootDir, r'ImportBackup\rws\bodempeilingen_historisch\ruweData')
asciiDataDir = os.path.join(rootDir, r'Import\rws\bodempeilingen_historisch\asciiData')
workdir = os.path.join(rootDir, r'ImportInterim\rws\bodempeilingen_historisch\tmpData')
y = os.path.join(rootDir, r'Config\MapLayerFiles\Achtergrond_polygonen//Achtergrond_polygonen.shp')
logFile = os.path.join(rootDir, r'ImportInterim\rws\bodempeilingen_historisch//log_file.out')
xmldir = os.path.join(rootDir, r'ImportInterim\rws\bodempeilingen_historisch\XMLGenerated')

In [22]:
xmldir

'D:\\OMS_Waddenzee\\trunk\\fews\\ImportInterim\\rws\\bodempeilingen_historisch\\XMLGenerated'

In [ ]:
# Set INPUT parameters
# ruweDataDir    :: input raw survey data
# asciiDataDir   :: output folder
# workDir        :: working directory (emptied each run)
# bgShp          :: background polygon shapefile of baggervakken
# logFile        :: file used to store logs

#ruweDataDir = r'D:\Projects\Pr\3317.20\BodempeilingScript_v4\survey_ruweData'
ruweDataDir = r'D:\Projects\Pr\3317.20\BodempeilingScript_v5\Import\rws\bodempeilingen_historisch\ruweData'
ruweDataDirBackup = r'D:\Projects\Pr\3317.20\BodempeilingScript_v5\ImportBackup\rws\bodempeilingen_historisch\ruweData'
asciiDataDir = r'D:\Projects\Pr\3317.20\BodempeilingScript_v5\Import\rws\bodempeilingen_historisch\asciiData'
workdir = r'D:\Projects\Pr\3317.20\BodempeilingScript_v5\ImportInterim\rws\bodempeilingen_historisch\tmpData'
y = r'D:\Projects\Pr\3317.20\BodempeilingScript_v5\Config\MapLayerFiles//Achtergrond_polygonen.shp'
logFile = r'D:\Projects\Pr\3317.20\BodempeilingScript_v5\ImportInterim\rws\bodempeilingen_historisch//log_file.out'

In [ ]:
# Set path OGR/GDAL files 
# ogr2ogr        :: converts simple features data between file formats
# gdalwarp       :: image reprojection and warping utility
# gdal_rasterize :: burns vector geometries into a raster
# gdal_translate :: converts raster data between different formats
# gdalbuildvrt   :: builds a VRT from a list of datasets
# gdalinfo       :: lists information about a raster dataset
# ogrinfo        :: lists information about an OGR supported data source

ogr2ogr = r'C:\Python35\Lib\site-packages\osgeo//ogr2ogr.exe'
gdalwarp = r'C:\Python35\Lib\site-packages\osgeo//gdalwarp.exe'
gdal_rasterize = r'C:\Python35\Lib\site-packages\osgeo//gdal_rasterize.exe'
gdal_translate = r'C:\Python35\Lib\site-packages\osgeo//gdal_translate.exe'
gdalbuildvrt = r'C:\Python35\Lib\site-packages\osgeo//gdalbuildvrt.exe'
gdalinfo = r'C:\Python35\Lib\site-packages\osgeo\gdalinfo.exe'
ogrinfo = r'C:\Python35\Lib\site-packages\osgeo\ogrinfo.exe'

In [ ]:
logger = logging.getLogger('survey2arcinfoascii')
hdlr = logging.FileHandler(logFile)
formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
hdlr.setFormatter(formatter)
logger.addHandler(hdlr) 
logger.setLevel(logging.INFO)

In [ ]:
logger.info ('Empty tmp dir %s' % (workdir))
filelist = glob.glob(workdir+'//*')
for f in filelist:
   os.remove(f)

In [ ]:
for root, dirs, files in os.walk(ruweDataDir):
    for file in files:
        if file.endswith('.asc'):
            print (file)  
            
            init_file = os.path.join( os.path.abspath(root), file )
            base, extension = os.path.splitext(file)
            logger.info (init_file)
            
            # 0 get timestamp
            tmod = os.path.getmtime(init_file)            
            t = time.strftime('%Y%m%d%H%M%S', time.localtime(int(tmod)))
#           print (tmod.strftime("%Y%m%d%H%M%S"))            
            #dateISO = datetime(int('20'+base[15:17]), int(base[17:19]), int(base[19:21]))
            #t = dateISO.strftime("%Y%m%d%H%M%S")
            logger.info (t)            
            
            # 1 convert survey data to CSV format
            a = os.path.join(workdir,t+'.csv')
            df = pd.read_csv(init_file, header=None)
            df.to_csv(a, header=False, index=False, sep=';')
            
            # 2 build OGRVRT from CSV file
            b = os.path.join(workdir,t+'.vrt')
            with open(b, 'w') as the_file:
                the_file.write(writeOGRVRT(t, a))
            
            # 2.1 get Extent from OGRVRT            
            command = ogrinfo+' -so ' + b + ' ' + t + ' | find "Extent"'
            logger.info (command)
            norm = sp.Popen(command, stdout=sp.PIPE, shell=True).communicate()            
            logger.info (norm)
            extent = non_decimal.sub('', str(norm[0])).replace('-',',')
            bb = [x.strip() for x in extent.split(',')]
            
            # 2.2 spatial query extent feature achtergrond SHP
            try:
                z = os.path.join(workdir,t+'bg_sel.shp')
                command = [ogr2ogr, z, y,'-spat',bb[0],bb[1],bb[2],bb[3]]
                logger.info (sp.list2cmdline(command))
                norm = sp.Popen(sp.list2cmdline(command),stdout=sp.PIPE, shell=True)
                norm.communicate()                      
            except Exception as e:
                logger.info (e)
                continue
                                
            # 3 create RASTER from OGRVRT
            c = os.path.join(workdir,t+'.tif')
            command = [gdal_rasterize, '-a','field_3','-tr','1.0','1.0', '-l',t,b,c]
            logger.info (sp.list2cmdline(command))
            norm = sp.Popen(sp.list2cmdline(command),stdout=sp.PIPE, shell=True)
            norm.communicate()

            try:
                ds, layer, z_features = get_features(z)
            except Exception as e:
                logger.info (e)
                continue
                
            for obID in (z_features):
                logger.info (obID)
                
                # 4 clip point RASTER with feature achtergrond SHP
                d = os.path.join(workdir,'grid'+str(obID).zfill(3)+'_'+t+'.tif.vrt')
                command = [gdalwarp, '-srcnodata', '0', '-dstnodata', '-9999', '-overwrite','-of', 'VRT', '-crop_to_cutline', 
                           '-cutline', z, '-cwhere', 'OBJECTID = '+str(obID), c, d]            
                logger.info (sp.list2cmdline(command))
                norm = sp.Popen(sp.list2cmdline(command),stdout=sp.PIPE, shell=True)
                norm.communicate()
                
                # 5A convert feature achtergrond SHP to RASTER
                e = os.path.join(workdir,'grid'+str(obID).zfill(3)+'_'+t+'_bg_tmp'+'.tif')
                command = [gdal_rasterize, '-a', 'NoDataValu', '-a_srs', 'EPSG:28992', '-where', 'OBJECTID = '+str(obID), 
                           '-tr', '1.0', '1.0', '-l', layer.GetDescription(), z, e]         
                logger.info (sp.list2cmdline(command))
                norm = sp.Popen(sp.list2cmdline(command),stdout=sp.PIPE, shell=True)
                norm.communicate()
                
                # 5B clip achtergrond RASTER
                f = os.path.join(workdir,'grid'+str(obID).zfill(3)+'_'+t+'_bg'+'.tif.vrt')
                command = [gdalwarp, '-srcnodata', '-9999', '-dstnodata', '-9999', '-of', 'VRT', '-tr', '1.0', '1.0', 
                           '-overwrite', '-crop_to_cutline', '-cutline', z, '-cwhere', 'OBJECTID = '+str(obID), e, f]            
                logger.info (sp.list2cmdline(command))
                norm = sp.Popen(sp.list2cmdline(command),stdout=sp.PIPE, shell=True)
                norm.communicate()                                

                # 6 Build VRT data source of point RASTER and feature achtergrond RASTER                
                g = os.path.join(workdir,'grid'+str(obID).zfill(3)+'_'+t+'.vrt')
                command = [gdalbuildvrt, '-srcnodata', '-9999', g, d, f]
                logger.info (sp.list2cmdline(command))
                norm = sp.Popen(sp.list2cmdline(command),stdout=sp.PIPE, shell=True)
                norm.communicate()

                # 7 Convert VRT to ArcInfoASCII
                h = os.path.join(asciiDataDir,'grid'+str(obID).zfill(3)+'_'+t+'.asc')
                command = [gdal_translate, '-of', 'AAIGrid', '-tr', '1.0', '1.0', g, h]
                logger.info (sp.list2cmdline(command))
                norm = sp.Popen(sp.list2cmdline(command),stdout=sp.PIPE, shell=True)
                norm.communicate()
                
                # 8 Only keep the ArcInfoASCIIs that contains data
                command = gdalinfo + ' -mm ' + h + ' | find "Computed"'
                logger.info (command)
                norm = sp.Popen(command, stdout=sp.PIPE, shell=True).communicate()
                logger.info (norm)
                if len(norm[0]) == 0:                    
                    for fl in glob.glob(h[0:-4]+'*'):
                        os.remove(fl)
            
            # 9 Move file to ImportBackup
            backup_file = os.path.join( os.path.abspath(ruweDataDirBackup), file )
            os.rename(init_file, backup_file)